In [72]:
import sys, os
from typing import DefaultDict

In [1]:

def parse_bw(filepath, target):
    if target < 0 or target > 5:
        print('Wrong target range!')
        sys.exit(1)
        
    """
    target
    0: msg_size
    1: iter
    2: bw_peak
    3: bw_avg
    4: msg_rate
    5: all
    """
    res = []
    
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            # (msg_size, iteration, bw_peak, bw_avg, msg_rate)
            if target == 5:
                row = []
                row.append(line[0])
                row.append(line[1])
                row.append(line[2])
                row.append(line[3])
                row.append(line[4])
                res.append(row)
            else:
                res.append(line[target])
        except:
            curline += 1
            continue
        curline += 1
    
    return res

def parse_lat(filepath):
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            return line
                
        except:
            curline += 1
            continue

In [4]:

# Output format = [[ data1 ], [data2], ... , columns = [] ]
dataset = []
columns = ['rnic', 'test_node', 'test_type', 'tx_depth', 'mtu', 'bg_msgsize', 'bg_bw', 'msgsize_tput', 'iter_tput', 'bw_peak_tput', 'bw_avg_tput', 'msgrate_tput', 'msgsize_lat', 'iter_lat', 't_min', 't_max', 't_typical', 't_avg', 't_stdev', '99th_percentile', '99.9th_percentile']

###################################
## Modify Here
TARGET = 'test_result_taskset_w2'
OUTPUT = 'X6_ETH100_taskset_w2.csv'
###################################

target_dir = './' + TARGET
if os.path.isdir(target_dir) == False:
    print("Directory is not valid!")
    sys.exit()
    
file_list = os.listdir(target_dir)

### Get target files ###
target_list = []
bw_t = ['rb', 'wb', 'sb']
for target in file_list:
    splt_target = target.split('_')
    if splt_target[2] in bw_t and splt_target[6] == 'tf':
        target_list.append(target)

for t in target_list:
    print(t)
    row = []
    splt_t = t.split('_')
    
    # add testing env
    row.append(splt_t[0])
    row.append(splt_t[1])
    row.append(splt_t[2])
    row.append(splt_t[3][1:])
    row.append(splt_t[4][1:])
    row.append(splt_t[5][2:])
    
    # add bg_bw
    bws = parse_bw(target_dir + '/' + t[:-2] + 'bf', 3)
    row.append(bws[int(len(bws)/2)])
    print(row)
    # add tput data
    tput_data = parse_bw(target_dir + '/' + t, 5)[0]
    for e in tput_data:
        row.append(e)
    
    # add latency data
    try:
        lat_data = parse_lat(target_dir + '/' + t.split('b_')[0] + 'l_' + t.split('b_')[1])
    except:
        print('File Not Exist: ' + target_dir + '/' + t.split('b_')[0] + 'l_' + t.split('b_')[1])
        lat_data = [None] * 9
    
    for e in lat_data:
        row.append(e)

    # print(row)
    dataset.append(row)
    break

#print(dataset)

X6ETH100_Y301_wb_t128_m1024_bg1024_tf
['X6ETH100', 'Y301', 'wb', '128', '1024', '1024', '9307.31']


In [75]:
import pandas as pd

df = pd.DataFrame(dataset, columns=columns)

print(df)
df.to_csv(OUTPUT)

         rnic test_node test_type tx_depth   mtu  bg_msgsize     bg_bw  \
0    X6ETH100      Y301        wb      128  1024        1024   9288.93   
1    X6ETH100      Y301        wb      128  1024     1048576  10820.53   
2    X6ETH100      Y301        wb      128  1024  1073741824  10456.14   
3    X6ETH100      Y301        wb      128  1024      131072  10784.72   
4    X6ETH100      Y301        wb      128  1024       16384  10784.04   
..        ...       ...       ...      ...   ...         ...       ...   
163  X6ETH100      Y301        wb        2   512        4096   2165.29   
164  X6ETH100      Y301        wb        2   512         512    331.83   
165  X6ETH100      Y301        wb        2   512      524288  10030.66   
166  X6ETH100      Y301        wb        2   512       65536  10080.47   
167  X6ETH100      Y301        wb        2   512        8192   3653.15   

    msgsize_tput  iter_tput bw_peak_tput  ... msgrate_tput msgsize_lat  \
0             16  100000000         0